# Métricas do reprocessamento

Código feito por Igor Laltuf, ago/23

---

## Objetivo

Analisar o impacto do reprocessamento das viagens da linha 663 entre 01/06/2022 até 30/04/2023

## Etapas
Comparar valores pré e pós reprocessamento segundo as seguintes variáveis:
- A quantidade de viagens completas
- Quilometragem total de viagens completas
- Valor do subsídio a ser pago
- Explicações pontuais sobre os resultados encontrados

## Conclusão

REESCREVER!!!!!

Sobre o período entre 01/06/2022 e 30/04/2023:

1) A linha 010 não teve viagens planejadas no mês de junho de 2022.

2) A quilometragem total das viagens completas aumentou de 66.790,1 km para 81.485,2 km, um aumento de 22% após o reprocessamento.

3) A quantidade de viagens identificadas aumentou de 6.414 para 15.670 viagens. Como as viagens que antes eram circulares foram divididas em duas viagens (ida e volta), era esperado que a quantidade de viagens pelo menos dobrasse, caso nenhuma nova viagem fosse identificada.

4) O valor do subsídio antes do reprocessamento era de R$ 100.756,68 e após o reprocessamento foi de R$ 169.626,86. Vale destacar que o valor do reprocessamento foi calculado sem os descontos por km.

### Carregar bibliotecas

In [2]:
#!pip install matplotlib
import basedosdados as bd
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import folium
from datetime import timedelta, datetime
from shapely import wkt

# Paths
import os
import sys
from pathlib import Path 

current_path = Path().resolve()

paths = dict()
paths["raw"] = current_path / 'data' / 'raw'
paths["treated"] = current_path / 'data' / 'treated'

for path in paths.values():
    if not os.path.exists(path):
        os.makedirs(path)

# Seta o projeto a ser cobrado
bd.config.billing_project_id = "rj-smtr-dev"

# Seta as configurações a serem usadas
bd.config.project_config_path = "C:/Users/igorl/.basedosdados/"
# bd.config.project_config_path = "D:\\basedosdados\\dev"

### 1. Comparar a variação na km após o reprocessamento

In [3]:
q = f"""
SELECT
  FORMAT_DATETIME('%Y-%m', DATA) AS mes_ano,
  SUM(ROUND(distancia_planejada,2)) AS distancia_total
FROM
  `rj-smtr.projeto_subsidio_sppo.viagem_completa`
WHERE
  DATA BETWEEN "2022-06-01"
  AND "2023-04-30"
  AND servico_informado = "663"
GROUP BY
  mes_ano
"""

viagens_realizadas_663 = bd.read_sql(q, from_file=True)

viagens_realizadas_663.to_csv('data/treated/viagens_realizadas_663_jun_22-abril_23.csv', index=False)

Downloading: 100%|██████████| 11/11 [00:00<00:00, 25.17rows/s]


In [4]:
q = f"""
SELECT
  FORMAT_DATETIME('%Y-%m', DATA) AS mes_ano,
  SUM(ROUND(distancia_planejada,2)) AS distancia_total
FROM
  `rj-smtr-dev.SMTR202212006620_reprocessamento.viagem_completa`
WHERE
  DATA BETWEEN "2022-06-01"
  AND "2023-04-30"
GROUP BY
  mes_ano
"""
viagens_reprocessadas_663 = bd.read_sql(q, from_file=True)

viagens_reprocessadas_663.to_csv('data/treated/viagens_reprocessadas_663_jun_22-abril_23.csv', index=False)

Downloading: 100%|██████████| 11/11 [00:00<00:00, 24.77rows/s]


In [5]:
viagens_reprocessadas_663 = pd.read_csv('data/treated/viagens_reprocessadas_663_jun_22-abril_23.csv')
viagens_realizadas_663 = pd.read_csv('data/treated/viagens_realizadas_663_jun_22-abril_23.csv')

km_mes_ano = pd.merge(viagens_realizadas_663, viagens_reprocessadas_663, how = 'left', on='mes_ano', suffixes=['','_reprocessada'])
km_mes_ano = km_mes_ano.sort_values('mes_ano')
km_mes_ano

,mes_ano,distancia_total,distancia_total_reprocessada
10,2022-06,887.04,1305.13
2,2022-07,604.05,971.08
7,2022-08,1249.92,1481.77
4,2022-09,1249.92,1543.08
6,2022-10,766.08,1091.89
8,2022-11,564.48,952.78
9,2022-12,1008.00,1286.83
5,2023-01,1207.80,1372.85
3,2023-02,1087.02,1245.63
0,2023-03,1449.36,1654.74


In [6]:
distancia_total_pre_reprocessamento = km_mes_ano['distancia_total'].sum()
distancia_total_pos_reprocessamento = km_mes_ano['distancia_total_reprocessada'].sum()
print("Comparação da quilometragem das viagens completas")
print("Valor total da km pré-reprocessamento:", distancia_total_pre_reprocessamento)
print("Valor total da km pós-reprocessamento:", distancia_total_pos_reprocessamento)
print("Aumento de ",round((distancia_total_pos_reprocessamento / distancia_total_pre_reprocessamento - 1) * 100, 2),"%")

Comparação da quilometragem das viagens completas
Valor total da km pré-reprocessamento: 11321.730000000001
Valor total da km pós-reprocessamento: 14293.26
Aumento de  26.25 %


In [7]:
# Criar o gráfico de linha
fig = go.Figure()

# Adicionar a linha com pontos para km_mes_ano['distancia_total']
fig.add_trace(go.Scatter(x=km_mes_ano['mes_ano'], y=km_mes_ano['distancia_total'],
                         mode='lines+markers',
                         name='Km pré-reprocessamento',
                         marker=dict(color='dodgerblue'),
                         opacity=0.75))

# Adicionar a linha com pontos para km_mes_ano['distancia_total_reprocessada']
fig.add_trace(go.Scatter(x=km_mes_ano['mes_ano'], y=km_mes_ano['distancia_total_reprocessada'],
                         mode='lines+markers',
                         name='Km reprocessada',
                         marker=dict(color='crimson'),
                         opacity=0.75))

# Configurar layout e outros parâmetros do gráfico
fig.update_layout(
    title=dict(text="Quilometragem apurada por mês - Serviço: 663", font=dict(color='black')),
    showlegend=False,
    width=800,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    # Adicionando anotações para o subtítulo com cores específicas
    annotations=[
        dict(
            x=-0.065,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='Valores',
            showarrow=False,
            font=dict(color='black', size=14)
        ),
        dict(
            x=0.025,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='pré-reprocessamento',
            showarrow=False,
            font=dict(color='dodgerblue', size=14)
        ),
        dict(
            x=0.27,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='e',
            showarrow=False,
            font=dict(color='black', size=14)
        ),
        dict(
            x=0.29,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='pós-reprocessamento.',
            showarrow=False,
            font=dict(color='crimson', size=14)
        )
    ]
)

fig.show()

### 2. Comparar a variação no valor do subsídio após o reprocessamento

2.1 Valor recebido pelos consórcios antes do reprocessamento

In [8]:
q = f"""
SELECT
  CONCAT(CAST(EXTRACT(YEAR FROM DATA) AS STRING), '-', LPAD(CAST(EXTRACT(MONTH FROM DATA) AS STRING), 2, '0')) AS mes_ano,
  SUM(viagens) as viagens,
  SUM(valor_subsidio_pago) AS valor_subsidio
FROM
  `rj-smtr.dashboard_subsidio_sppo.sumario_servico_dia_historico`
WHERE
  servico = '663'
  AND DATA BETWEEN '2022-06-01' AND '2023-04-30'
GROUP BY
  mes_ano
"""
subsidio_pago_mes = bd.read_sql(q, from_file=True)


Downloading: 100%|██████████| 11/11 [00:00<00:00, 29.73rows/s]


In [9]:
subsidio_pago_mes = subsidio_pago_mes.sort_values('mes_ano')
subsidio_pago_mes = subsidio_pago_mes[['mes_ano', 'viagens', 'valor_subsidio']]
subsidio_pago_mes

,mes_ano,viagens,valor_subsidio
8,2022-06,22,1288.10
5,2022-07,15,889.26
0,2022-08,31,1306.26
4,2022-09,31,1834.43
3,2022-10,19,783.72
6,2022-11,14,308.42
9,2022-12,25,1291.68
2,2023-01,30,3031.06
1,2023-02,27,2489.19
10,2023-03,36,3394.35


2.2 Valor que deve ser pago após o reprocessamento

Com base nas viagens identificadas na tabela de viagem completa que foi reprocessada, foram verificados aqueles dias em que POD >= 80%, para então calcular o valor do subsídio de acordo com os valores por km da tabela `subsidio_data_versao_efetiva` que está no dataset `rj-smtr-dev.SMTR202212006620_reprocessamento`.

Vale destacar que esta tabela exclui as glosas de 2023 do valor do subsídio por km. Ex: considera 2.81 por km após 16 de janeiro de 2023. 

In [10]:
q = f"""
WITH
  viagem_dia AS (
  SELECT
    DATA,
    COUNT(*) AS viagens,
    SUM(distancia_planejada) AS km_apurada
  FROM
    `rj-smtr-dev.SMTR202212006620_reprocessamento.viagem_completa`
  WHERE
    DATA BETWEEN "2022-06-01"
    AND "2023-04-30"
  GROUP BY
    DATA
  ),
  viagem_planejada AS ( -- pegar a km planejada para o dia
  SELECT DISTINCT 
    DATA,
    distancia_total_planejada
  FROM
    `rj-smtr.projeto_subsidio_sppo.viagem_planejada`
  WHERE
    servico = '663'
    AND DATA BETWEEN "2022-06-01"
    AND "2023-04-30"
  ),
  tabela_pod AS (
  SELECT
    *,
  ROUND(CASE WHEN distancia_total_planejada = 0 THEN 0 ELSE km_apurada / distancia_total_planejada END, 2) AS perc_km_planejada
  FROM
    viagem_dia vd
  RIGHT JOIN
    viagem_planejada vp
  USING
    (DATA) 
  ),
  tabela_subsidio_km AS (
  SELECT
    DATA,
    valor_subsidio_por_km
  FROM
    `rj-smtr-dev.SMTR202212006620_reprocessamento.subsidio_data_versao_efetiva` )
SELECT
  DATA,
  COALESCE(viagens, 0) as viagens,
  COALESCE(km_apurada,0) as km_apurada,
  COALESCE(distancia_total_planejada,0) as distancia_total_planejada,
  COALESCE(perc_km_planejada,0) as perc_km_planejada,
  COALESCE(valor_subsidio_por_km,0) as valor_subsidio_por_km,
   CASE 
   WHEN perc_km_planejada >= 0.8 THEN valor_subsidio_por_km * km_apurada
   ELSE 0
   END as valor_subsidio
FROM
  tabela_pod
LEFT JOIN
  tabela_subsidio_km
USING
  (DATA)
"""

subsidio_a_pagar_dia = bd.read_sql(q, from_file=True)
subsidio_a_pagar_dia = subsidio_a_pagar_dia.sort_values('DATA')
subsidio_a_pagar_dia

Downloading: 100%|██████████| 334/334 [00:00<00:00, 856.42rows/s]


,DATA,viagens,km_apurada,distancia_total_planejada,perc_km_planejada,valor_subsidio_por_km,valor_subsidio
304,2022-06-01,13,266.341,80.633,3.30,2.13,567.30633
209,2022-06-02,12,241.626,80.633,3.00,2.13,514.66338
56,2022-06-03,11,216.911,80.633,2.69,2.13,462.02043
280,2022-06-04,0,0.000,50.395,0.00,2.13,0.00000
137,2022-06-05,0,0.000,40.316,0.00,2.13,0.00000
...,...,...,...,...,...,...,...
244,2023-04-26,4,80.542,80.520,1.00,2.81,226.32302
202,2023-04-27,4,80.542,80.520,1.00,2.81,226.32302
45,2023-04-28,4,80.542,80.520,1.00,2.81,226.32302
184,2023-04-29,0,0.000,0.000,0.00,2.81,0.00000


In [11]:
subsidio_a_pagar_dia['DATA'] = pd.to_datetime(subsidio_a_pagar_dia['DATA'])
subsidio_a_pagar_dia['mes_ano'] = subsidio_a_pagar_dia['DATA'].dt.strftime('%Y-%m')
subsidio_a_pagar_mes = subsidio_a_pagar_dia[['mes_ano', 'viagens', 'valor_subsidio']]
subsidio_a_pagar_mes = subsidio_a_pagar_mes.groupby('mes_ano').sum().reset_index()  
subsidio_a_pagar_mes

,mes_ano,viagens,valor_subsidio
0,2022-06,68,1543.99014
1,2022-07,53,934.65928
2,2022-08,77,1304.78040
3,2022-09,78,2243.98125
4,2022-10,59,782.86824
5,2022-11,53,331.87383
6,2022-12,68,1290.28284
7,2023-01,70,3214.21154
8,2023-02,63,2785.32539
9,2023-03,84,3394.84530


O aumento do valor do subsídio não é proporcional ao aumento da quantidade de viagens, pois com o reprocessamento, cada viagem circular virou duas viagens separadas, uma de ida e outra de volta.

Isto quer dizer que o esperado era que a quantidade de viagens pelo menos dobrasse após o reprocessamento, mesmo que não fosse identificada nenhuma viagem nova.

In [12]:
# juntar as duas tabelas
tabela_subsidio_mes = pd.merge(subsidio_pago_mes, subsidio_a_pagar_mes, how='left', on='mes_ano', suffixes=['_pre','_pos'])
tabela_subsidio_mes

,mes_ano,viagens_pre,valor_subsidio_pre,viagens_pos,valor_subsidio_pos
0,2022-06,22,1288.10,68,1543.99014
1,2022-07,15,889.26,53,934.65928
2,2022-08,31,1306.26,77,1304.78040
3,2022-09,31,1834.43,78,2243.98125
4,2022-10,19,783.72,59,782.86824
5,2022-11,14,308.42,53,331.87383
6,2022-12,25,1291.68,68,1290.28284
7,2023-01,30,3031.06,70,3214.21154
8,2023-02,27,2489.19,63,2785.32539
9,2023-03,36,3394.35,84,3394.84530


In [13]:
tabela_subsidio_mes
total_subsidio_comparacao = tabela_subsidio_mes[['viagens_pre', 'viagens_pos','valor_subsidio_pre','valor_subsidio_pos']].sum(axis=0)
total_subsidio_comparacao

viagens_pre             280.00000
viagens_pos             741.00000
valor_subsidio_pre    19558.24000
valor_subsidio_pos    21159.97558
dtype: float64

In [14]:
# Criar o gráfico de linha com plotly
fig = go.Figure()

# Adicionar a linha com pontos para tabela_subsidio_mes['viagens_pre']
fig.add_trace(go.Scatter(x=tabela_subsidio_mes['mes_ano'], y=tabela_subsidio_mes['viagens_pre'],
                         mode='lines+markers',
                         name='Qtd de viagens pré-reprocessamento',
                         marker=dict(color='dodgerblue'),
                         opacity=0.75))

# Adicionar a linha com pontos para tabela_subsidio_mes['viagens_pos']
fig.add_trace(go.Scatter(x=tabela_subsidio_mes['mes_ano'], y=tabela_subsidio_mes['viagens_pos'],
                         mode='lines+markers',
                         name='Qtd de viagens reprocessadas',
                         marker=dict(color='crimson'),
                         opacity=0.75))

# Configurar layout e outros parâmetros do gráfico seguindo o design anterior
fig.update_layout(
    title=dict(text="Quantidade de viagens por mês - Serviço: 663", font=dict(color='black')),
    showlegend=False,
    width=800,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    # Adicionando anotações para o subtítulo com cores específicas
    annotations=[
        dict(
            x=-0.065,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='Valores',
            showarrow=False,
            font=dict(color='black', size=14)
        ),
        dict(
            x=0.025,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='pré-reprocessamento',
            showarrow=False,
            font=dict(color='dodgerblue', size=14)
        ),
        dict(
            x=0.27,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='e',
            showarrow=False,
            font=dict(color='black', size=14)
        ),
        dict(
            x=0.29,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='pós-reprocessamento.',
            showarrow=False,
            font=dict(color='crimson', size=14)
        )
    ]
)

fig.show()

In [15]:
# Criar o gráfico de linha com plotly
fig = go.Figure()

# Adicionar a linha com pontos para tabela_subsidio_mes['valor_subsidio_pre']
fig.add_trace(go.Scatter(x=tabela_subsidio_mes['mes_ano'], y=tabela_subsidio_mes['valor_subsidio_pre'],
                         mode='lines+markers',
                         name='Valor do subsídio pré-reprocessamento',
                         marker=dict(color='dodgerblue'),
                         opacity=0.75))

# Adicionar a linha com pontos para tabela_subsidio_mes['valor_subsidio_pos']
fig.add_trace(go.Scatter(x=tabela_subsidio_mes['mes_ano'], y=tabela_subsidio_mes['valor_subsidio_pos'],
                         mode='lines+markers',
                         name='Valor do subsídio reprocessado',
                         marker=dict(color='crimson'),
                         opacity=0.75))

# Configurar layout e outros parâmetros do gráfico seguindo o design anterior
fig.update_layout(
    title=dict(text="Valor do subsídio por mês - Serviço: 663", font=dict(color='black')),
    showlegend=False,
    width=800,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    # Adicionando anotações para o subtítulo com cores específicas
    annotations=[
        dict(
            x=-0.065,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='Valores',
            showarrow=False,
            font=dict(color='black', size=14)
        ),
        dict(
            x=0.025,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='pré-reprocessamento',
            showarrow=False,
            font=dict(color='dodgerblue', size=14)
        ),
        dict(
            x=0.27,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='e',
            showarrow=False,
            font=dict(color='black', size=14)
        ),
        dict(
            x=0.29,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='pós-reprocessamento.',
            showarrow=False,
            font=dict(color='crimson', size=14)
        )
    ]
)

fig.show()

### 3. Análise do POD


3.1 POD pré-reprocessamento

In [16]:
# POD pré-reprocessamento
q = f"""
SELECT
  data, viagens, perc_km_planejada
FROM
  `rj-smtr.dashboard_subsidio_sppo.sumario_servico_dia_historico`
WHERE
  servico = '663'
  AND DATA BETWEEN '2022-06-01' AND '2023-04-30'
"""
pod_pre_reprocessamento = bd.read_sql(q, from_file=True)

Downloading: 100%|██████████| 295/295 [00:00<00:00, 817.16rows/s]


In [17]:
# Assegurando que a coluna 'data' seja do tipo datetime
pod_pre_reprocessamento = pod_pre_reprocessamento.sort_values('data')
pod_pre_reprocessamento['data'] = pd.to_datetime(pod_pre_reprocessamento['data'])
# Criando a nova coluna 'mes_ano'
pod_pre_reprocessamento['mes_ano'] = pod_pre_reprocessamento['data'].dt.to_period('M')
pod_pre_reprocessamento['mes_ano'] = pod_pre_reprocessamento['mes_ano'].astype(str)
pod_pre_reprocessamento

,data,viagens,perc_km_planejada,mes_ano
232,2022-06-01,6,300.00,2022-06
219,2022-06-02,5,250.00,2022-06
276,2022-06-03,4,200.00,2022-06
240,2022-06-04,0,0.00,2022-06
208,2022-06-05,0,0.00,2022-06
...,...,...,...,...
61,2023-04-24,2,100.01,2023-04
11,2023-04-25,2,100.01,2023-04
43,2023-04-26,2,100.01,2023-04
39,2023-04-27,2,100.01,2023-04


In [18]:

y = "perc_km_planejada"
title = f"Distribuição do percentual de operação diária (POD) por mês - Serviço: 663"

fig = px.box(
    pod_pre_reprocessamento, 
    x="mes_ano", 
    y=y, 
    title=title,
    template="plotly_white",
    labels={
        "mes_ano": "",
        y: ""
    },
    points="all"
)

fig.update_layout(
    yaxis_range=[-10, 200],  
    yaxis_ticksuffix="%",
    width=800,
    height=600,
    showlegend=False,
)

fig.add_hline(y=80, annotation_text="min = 80%")  # Linha horizontal para indicar o mínimo de 80%

fig.show()


3.2 POD pós-reprocessamento

In [19]:
pod_pos_reprocessamento = subsidio_a_pagar_dia[["mes_ano","viagens","perc_km_planejada"]]
pod_pos_reprocessamento['perc_km_planejada'] = pod_pos_reprocessamento['perc_km_planejada'] * 100
pod_pos_reprocessamento

,mes_ano,viagens,perc_km_planejada
304,2022-06,13,330.0
209,2022-06,12,300.0
56,2022-06,11,269.0
280,2022-06,0,0.0
137,2022-06,0,0.0
...,...,...,...
244,2023-04,4,100.0
202,2023-04,4,100.0
45,2023-04,4,100.0
184,2023-04,0,0.0


In [20]:

y = "perc_km_planejada"
title = "Distribuição do percentual de operação diária (POD) por mês - Serviço: 663 <br>Versão reprocessada"

fig = px.box(
    pod_pos_reprocessamento, 
    x="mes_ano", 
    y=y, 
    title=title,
    template="plotly_white",
    labels={
        "mes_ano": "",
        y: ""
    },
    points="all"
)

fig.update_layout(
    yaxis_range=[-50, 230],  
    yaxis_ticksuffix="%",
    width=800,
    height=600,
    showlegend=False,
)

fig.add_hline(y=80, annotation_text="min = 80%")  # Linha horizontal para indicar o mínimo de 80%

fig.show()


Consolidar POD pré e pós reprocessamento em apenas um gráfico.

In [21]:
# Calcular a média e o desvio padrão de perc_km_planejada por mês para ambos os DataFrames
media_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].mean()
std_pre = pod_pre_reprocessamento.groupby('mes_ano')['perc_km_planejada'].std()

media_pos = pod_pos_reprocessamento.groupby('mes_ano')['perc_km_planejada'].mean()
std_pos = pod_pos_reprocessamento.groupby('mes_ano')['perc_km_planejada'].std()

Com o reprocessamento, houve uma melhoria do valor do Percentual de Operação Diária.

In [22]:
maiores_perc_pos = subsidio_a_pagar_dia.sort_values(by='perc_km_planejada', ascending=False)
maiores_perc_pos

,DATA,viagens,km_apurada,distancia_total_planejada,perc_km_planejada,valor_subsidio_por_km,valor_subsidio,mes_ano
304,2022-06-01,13,266.341,80.633,3.30,2.13,567.30633,2022-06
209,2022-06-02,12,241.626,80.633,3.00,2.13,514.66338,2022-06
56,2022-06-03,11,216.911,80.633,2.69,2.13,462.02043,2022-06
170,2022-09-13,6,129.972,80.633,1.61,1.75,227.45100,2022-09
10,2022-09-02,4,80.542,50.395,1.60,1.75,140.94850,2022-09
...,...,...,...,...,...,...,...,...
260,2022-11-13,0,0.000,40.316,0.00,1.53,0.00000,2022-11
213,2022-11-19,0,0.000,50.395,0.00,1.53,0.00000,2022-11
133,2022-11-20,0,0.000,40.316,0.00,1.53,0.00000,2022-11
68,2022-11-26,0,0.000,50.395,0.00,1.53,0.00000,2022-11


In [23]:
# Criar o gráfico de linha
fig = go.Figure()

# Adicionar a linha com pontos e desvio padrão para pod_pre_reprocessamento
fig.add_trace(go.Scatter(x=media_pre.index, y=media_pre.values,
                    mode='lines+markers',
                    name='Antes do Reprocessamento',
                    marker=dict(color='dodgerblue'),
                    opacity=0.75,
                    error_y=dict(type='data', array=std_pre, visible=True, color='dodgerblue')  # Corrigindo a cor aqui
))

# Adicionar a linha com pontos e desvio padrão para pod_pos_reprocessamento
fig.add_trace(go.Scatter(x=media_pos.index, y=media_pos.values,
                    mode='lines+markers',
                    name='Após Reprocessamento',
                    marker=dict(color='crimson'),
                    opacity=0.75,
                    error_y=dict(type='data', array=std_pos, visible=True, color='crimson')  # Corrigindo a cor aqui
))

# Configurar layout e outros parâmetros do gráfico
fig.update_layout(
    title=dict(text="Percentual de Operação Diária (POD) por mês - Serviço: 663", font=dict(color='black')),
    showlegend=False,
    yaxis_range=[0, 230],
    yaxis_ticksuffix="%",
    width=800,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    # Adicionando anotações para o subtítulo com cores específicas
    annotations=[
        dict(
            x=-0.065,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='Valor médio e desvio-padrão mensais do POD',
            showarrow=False,
            font=dict(color='black', size=14)  # Aplicando a cor do título
        ),
        dict(
            x=0.445,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='antes do reprocessamento',
            showarrow=False,
            font=dict(color='dodgerblue', size=14)
        ),
        dict(
            x=0.745,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='e',
            showarrow=False,
            font=dict(color='black', size=14)  # Aplicando a cor do título
        ),
        dict(
            x=0.765,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='após o reprocessamento.',
            showarrow=False,
            font=dict(color='crimson', size=14)
        )
    ]
)

# Adicionar linha horizontal para indicar o mínimo de 80%
fig.add_hline(y=80, annotation_text="min = 80%")

fig.show()

In [24]:
# O que explica o POD caindo após 01-01-2023???

subsidio_a_pagar_dia['DATA'] = pd.to_datetime(subsidio_a_pagar_dia['DATA'])
subsidio_a_pagar_dia['mes_ano'] = subsidio_a_pagar_dia['DATA'].dt.strftime('%Y-%m')

POD_mes = subsidio_a_pagar_dia[['DATA', 'distancia_total_planejada', 'km_apurada']].sort_values(by='DATA')
POD_mes


# Criar o gráfico de linha com plotly
fig = go.Figure()

# Adicionar a linha com pontos para tabela_subsidio_mes['valor_subsidio_pre']
fig.add_trace(go.Scatter(x=POD_mes['DATA'], y=POD_mes['distancia_total_planejada'],
                         mode='lines',
                         name='Distancia Planejada',
                         marker=dict(color='dodgerblue'),
                         opacity=0.75))

# Adicionar a linha com pontos para tabela_subsidio_mes['valor_subsidio_pos']
fig.add_trace(go.Scatter(x=POD_mes['DATA'], y=POD_mes['km_apurada'],
                         mode='lines',
                         name='Distância Apurada',
                         marker=dict(color='crimson'),
                         opacity=0.75))

# Configurar layout e outros parâmetros do gráfico seguindo o design anterior
fig.update_layout(
    title=dict(text="Valor do subsídio por mês - Serviço: 663", font=dict(color='black')),
    showlegend=False,
    width=800,
    height=600,
    plot_bgcolor='white',
    xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False),
    # Adicionando anotações para o subtítulo com cores específicas
    annotations=[
        dict(
            x=-0.065,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='Valores',
            showarrow=False,
            font=dict(color='black', size=14)
        ),
        dict(
            x=0.025,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='pré-reprocessamento',
            showarrow=False,
            font=dict(color='dodgerblue', size=14)
        ),
        dict(
            x=0.27,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='e',
            showarrow=False,
            font=dict(color='black', size=14)
        ),
        dict(
            x=0.29,
            y=1.10,
            xref='paper',
            yref='paper',
            xanchor='left',
            text='pós-reprocessamento.',
            showarrow=False,
            font=dict(color='crimson', size=14)
        )
    ]
)

fig.show()



### 4. Exportar planilha com dados

Exportar planilha resumo com valores a pagar antes e depois, assim como a km identificada na planilha de viagens completas.

In [25]:
km_mes_ano_soma = km_mes_ano[['distancia_total', 'distancia_total_reprocessada']].sum()
km_mes_ano_soma = km_mes_ano_soma.to_frame().T

km_mes_ano_soma = km_mes_ano_soma.rename(columns={
    'distancia_total': 'distancia_total_pre',
    'distancia_total_reprocessada': 'distancia_total_pos'
})
km_mes_ano_soma


tabela_subsidio_mes_soma = tabela_subsidio_mes[['viagens_pre', 'viagens_pos','valor_subsidio_pre', 'valor_subsidio_pos']].sum()
tabela_subsidio_mes_soma = tabela_subsidio_mes_soma.to_frame().T
tabela_subsidio_mes_soma

km_mes_ano_soma['key'] = 1
tabela_subsidio_mes_soma['key'] = 1

tabela_comparacao_exportar = km_mes_ano_soma.merge(tabela_subsidio_mes_soma, on='key', how='left').drop('key', axis=1)

tabela_comparacao_exportar
tabela_comparacao_exportar.to_excel("data/output/comparacao_pre_pos_reprocessamento_663.xlsx", index=False, engine='openpyxl')